In [ ]:
import os
import sys
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize

sys.path.append(os.path.dirname(os.path.abspath('')))
import utils.prepare_data as pd
import utils.xyzuvd as xyzuvd
import utils.visual as visual
from utils.directory import DATA_DIR, DATASET_DIR

In [ ]:
train_pairs, test_pairs = pd.get_fpha_data_list(DATASET_DIR)
file_name = np.asarray([i for i,j in train_pairs])
xyz_gt = np.reshape([j for i,j in train_pairs], (-1, 21 , 3))
uvd_gt = xyzuvd.xyz2uvd_color(xyz_gt)
ccs_gt = xyzuvd.xyz2ccs_color(xyz_gt)
REORDER = visual.REORDER

In [ ]:
idx = 1774
file_name_i = file_name[idx]
xyz_gt_i = xyz_gt[idx]
uvd_gt_i = uvd_gt[idx]
ccs_gt_i = ccs_gt[idx]
img = Image.open(os.path.join(DATASET_DIR, 'First_Person_Action_Benchmark', 'Video_files', file_name_i))
img = np.asarray(img, dtype='uint32')

In [ ]:
fig, ax = plt.subplots()
ax.imshow(img)
visual.visualize_joints_2d(ax, uvd_gt_i[REORDER], joint_idxs=False)

In [ ]:
img_resize = resize(img, (416, 416), order=3, preserve_range=True).astype('uint32')
uvd_gt_i_resize = uvd_gt_i.copy()
uvd_gt_i_resize[:, 0] *= 416/1920
uvd_gt_i_resize[:, 1] *= 416/1080

In [ ]:
fig, ax = plt.subplots()
ax.imshow(img_resize)
visual.visualize_joints_2d(ax, uvd_gt_i_resize[REORDER], joint_idxs=False)

In [ ]:
pad = 10
x_max = int(np.amax(uvd_gt_i_resize[:, 0])) + pad
x_min = np.maximum(int(np.amin(uvd_gt_i_resize[:, 0])) - pad, 0)
y_max = int(np.amax(uvd_gt_i_resize[:, 1])) + pad
y_min = np.maximum(int(np.amin(uvd_gt_i_resize[:, 1])) - pad, 0)
z_max = int(np.amax(uvd_gt_i_resize[:, 2])) + pad
z_min = np.maximum(int(np.amin(uvd_gt_i_resize[:, 2])) - pad, 0)

In [ ]:
print(x_min, x_max)
print(y_min, y_max)
print(z_min, z_max)

In [ ]:
import math

x_min_scale = x_min//32
x_max_scale = math.ceil(x_max/32)
y_min_scale = y_min//32
y_max_scale = math.ceil(y_max/32)
z_min_scale = z_min//120
z_max_scale = math.ceil(z_max/120)

print(x_min_scale, x_max_scale)
print(y_min_scale, y_max_scale)
print(z_min_scale, z_max_scale)

In [ ]:
np.ravel_multi_index((0,1,0), (13,13,5))

In [ ]:
import itertools
comb = [list(i) for i in itertools.product(np.arange(x_min_scale, x_max_scale), np.arange(y_min_scale, y_max_scale))]
print(comb)

In [ ]:
import matplotlib.patches as patches
fig, ax = plt.subplots(figsize=(13,13))
ax.imshow(img_resize)
visual.visualize_joints_2d(ax, uvd_gt_i_resize[REORDER], joint_idxs=False)
for x,y in comb:
    rect = patches.Rectangle((x*32,y*32),32,32,linewidth=1,edgecolor='r', facecolor='r', fill=True, alpha=0.5)
    ax.add_patch(rect)

In [ ]:
comb_3d = [list(i) for i in itertools.product(np.arange(x_min_scale, x_max_scale), \
                                              np.arange(y_min_scale, y_max_scale), \
                                              np.arange(z_min_scale, z_max_scale))]
print(comb_3d)

In [ ]:
index = np.asarray(np.unravel_index(np.arange(845), (13, 13, 5)))

In [ ]:
ravel_comb = []
for comb in comb_3d:
    ravel_comb.append(np.ravel_multi_index(comb, (13,13,5)))
print(ravel_comb)

In [ ]:
all = np.zeros(845)
all[ravel_comb] = 1
all = all.astype('uint8')
print(all)

In [ ]:
test = np.ones(845)
test[all != 1] = 2
print(test)

In [ ]:
bbox_xyz, bbox_uvd = pd.get_bbox(260, 1000, 935.732544, 540.681030)
print(bbox_xyz)
print(bbox_uvd)